# Drive mount

In [ ]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install datasets
!pip install transformers[torch] -U
!pip install peft
!pip install bitsandbytes
!pip install trl

Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 11.2 MB/s eta 0:00:00


In [ ]:
import os
import json
import numpy as np
import pandas as pd
import re
import string
from collections import Counter
from tqdm import tqdm

import torch
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForCausalLM,
)
from datasets import load_dataset, Dataset, DatasetDict
from accelerate import Accelerator


# 데이터를 Colab 서버 내에 세팅

# Load Validation / Test dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/MyDrive/data/train.csv'
train_data = pd.read_csv(file_path)

train_data = train_data.sample(frac=1).reset_index(drop=True)

val_data=train_data[:10]
train_data = train_data[10:]

val_label_df = val_data[['question', 'answer']]

train_data.head(2)

,id,context,question,answer
10,TRAIN_16566,신한금융투자(대표이사 이영창)는 해외주식 스탁콘 판매량이 4주만에 4천건을 돌파했다...,가장 많은 매수 종목,테슬라
11,TRAIN_02187,NH-Amundi자산운용(대표이사 박학주)은 ‘100년 기업 그린 코리아 펀드’가 ...,출시한 지 얼마 만에 펀드 설정액을 넘어섰어,5개월


# Model Load

In [ ]:
# Quantized model
model_id = "sonhy02/INHA-DACON-8B"
model = AutoModelForCausalLM.from_pretrained(model_id,
                                            torch_dtype="auto") # 한정된 메모리에서 사용하기 위해, 4비트 정밀도로 로드


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/132k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

In [ ]:
# Model load + Lora
model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
base_model = AutoModelForCausalLM.from_pretrained(model_id,
                                                  torch_dtype=torch.float16,
                                                  device_map='auto')

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side='right'
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1



config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from peft import PeftModel
ADAPTER_MODEL = "sonhy02/INHA-DACON-5h"
model = PeftModel.from_pretrained(base_model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)
model = model.merge_and_unload()

adapter_config.json:   0%|          | 0.00/737 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/10.5M [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id) # 본 모델의 토크나이저 로드
tokenizer.use_default_system_prompt = False  # 기본 시스템 프롬프트를 사용하지 않도록 설정
accelerator = Accelerator()
model, tokenizer = accelerator.prepare(model, tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Define F1 score

In [ ]:
def normalize_answer(s):
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text)
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)
        return text

    def white_space_fix(text):
        '''연속된 공백일 경우 하나의 공백으로 대체'''
        return ' '.join(text.split())

    def remove_punc(text):
        '''구두점 제거'''
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        '''소문자 전환'''
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()

    # 문자 단위로 f1-score를 계산 합니다.
    prediction_Char = []
    for tok in prediction_tokens:
        now = [a for a in tok]
        prediction_Char.extend(now)

    ground_truth_Char = []
    for tok in ground_truth_tokens:
        now = [a for a in tok]
        ground_truth_Char.extend(now)

    common = Counter(prediction_Char) & Counter(ground_truth_Char)
    num_same = sum(common.values())
    if num_same == 0:
        return 0

    precision = 1.0 * num_same / len(prediction_Char)
    recall = 1.0 * num_same / len(ground_truth_Char)
    f1 = (2 * precision * recall) / (precision + recall)

    return f1

def evaluate(ground_truth_df, predictions_df):
    predictions = dict(zip(predictions_df['question'], predictions_df['answer']))
    f1 = exact_match = total = 0

    for index, row in ground_truth_df.iterrows():
        question_text = row['question']
        ground_truths = row['answer']
        total += 1
        if question_text not in predictions:
            continue
        prediction = predictions[question_text]
        f1 = f1 + f1_score(prediction, ground_truths)

    f1 = 100.0 * f1 / total
    return {'f1': f1}


# Validation
- 로드한 모델에 대해 간단한 프롬프트를 입력하여 성능 검증

In [ ]:
qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
# 모델로 추론 후, 전처리를 수행한 뒤, 완성된 정답으로 반환합니다.
def generate_response(question_prompt):
    # 생성할 최대 토큰 수와, 답변 생성 수, 패딩 토큰의 idx를 지정하여 모델 파이프 라인을 설정하고, 답변을 생성합니다.
    response = qa_pipeline(question_prompt, max_new_tokens=100, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)[0]['generated_text']
    if "Answer:" in response:
            # Answer: 이후에 생성된 토큰 들만을 답변으로 사용합니다.
            response = response.split("Answer:", 1)[1][:20]

            # 토큰 반복 생성 및 노이즈 토큰 관련 처리
            if "Que" in response:
                response = response.split("Que", 1)[0]
            if "⊙" in response:
                response = response.split("⊙", 1)[0]
            if "Con" in response:
                response = response.split("Con", 1)[0]
    return response

In [ ]:
#Google Colab T4 GPU 기준 10개 샘플 답안 생성에 1분 소요
predict_dict = {}
count = 0

for index, row in val_data.iterrows():
    try:
        context = row['context']
        question = row['question']


        if context is not None and question is not None:
            question_prompt = f"너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. \
                                Question에 대한 답변만 가급적 한 단어로 최대한 간결하게 답변하도록 해. \
                                Context: {context} Question: {question}\n Answer:"
            answer = generate_response(question_prompt)
            answer = normalize_answer(answer)
            predict_dict[question] = answer
        else:
            predict_dict[question] = 'Invalid question or context'

        print("Answer for question:", question, ":", predict_dict[question])
        count += 1
        print("Processed count:", count)
    except Exception as e:
        print(f"Error processing question {e}")



Answer for question: 강원도 전 지역에서 91%의 수출을 점한 지역이 어디야 : 원주 홍천 동해 춘천 강릉
Processed count: 1
Answer for question: 전항일 사장이 이베이재팬 대표로 취임한 연도 : 2018년
Processed count: 2
Answer for question: 샤르르의 편곡에 참가한 사람이 누구야 : doko 도코
Processed count: 3
Answer for question: 예비사회적기업 일자리창출사업 공모에 신청한 기업의 수 : 52개 기업
Processed count: 4
Answer for question: 어디서 우수 청년 창업자를 발굴해 다양한 지원을 해주는 사업을 시행해 : 청년창업사관학교
Processed count: 5
Answer for question: 어디에서 국내 우주개발 정책 수립의 걸림돌이 되는 추격형 연구를 탈피한 새로운 연구를 하는 거야 : 한국항공우주연구원
Processed count: 6
Answer for question: 한국새농민충북도회가 취약계층의 사람들을 위해 충북사회복지공동모금회에 전달한 건 뭐야 : 농산물 꾸러미 200상자
Processed count: 7
Answer for question: 어떤 사람들에게 금감원에서 맞춤형 지원을 한다는 거야 : 청년 자영업자에 대해선 지원을 특화
Processed count: 8
Answer for question: 충북지방중소기업벤처기업청에서 백년가게를 뽑아 돕는 이유는 뭐지 : 오랜 기간 노하우를 바탕으로 지속가
Processed count: 9
Answer for question: 충청지방통계청에 의하면 충북지역에서 일 년 동안 높아진 소비자물가지수는 얼마나 돼 : 260
Processed count: 10


In [ ]:
val_inf_df = pd.DataFrame(list(predict_dict.items()), columns=['question', 'answer'])
val_inf_df.head()

,question,answer
0,강원도 전 지역에서 91%의 수출을 점한 지역이 어디야,원주 홍천 동해 춘천 강릉
1,전항일 사장이 이베이재팬 대표로 취임한 연도,2018년
2,샤르르의 편곡에 참가한 사람이 누구야,doko 도코
3,예비사회적기업 일자리창출사업 공모에 신청한 기업의 수,52개 기업
4,어디서 우수 청년 창업자를 발굴해 다양한 지원을 해주는 사업을 시행해,청년창업사관학교


In [ ]:
# f1-score 를 출력합니다.
results = evaluate( val_inf_df, val_label_df)
print(results)

{'f1': 62.530103995621246}


# Inference

In [ ]:
file_path = '/content/drive/MyDrive/data/test.csv'
test_data = pd.read_csv(file_path)

In [ ]:
# 모델 추론
submission_dict = {}

for index, row in test_data.iterrows():
    try:
        context = row['context']
        question = row['question']
        id = row['id']

        if context is not None and question is not None:
            question_prompt = f"너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. \
                                Question에 대한 답변만 가급적 한 단어로 최대한 간결하게 답변하도록 해. \
                                Context: {context} Question: {question}\n Answer:"

            answer = generate_response(question_prompt)
            answer = normalize_answer(answer)
            submission_dict[id] = answer
        else:
            submission_dict[id] = 'Invalid question or context'

    except Exception as e:
        print(f"Error processing question {e}")

# Submission

In [ ]:
# 제출
df = pd.DataFrame(list(submission_dict.items()), columns=['id', 'answer'])
df.to_csv( '/content/drive/MyDrive/submission.csv', index=False)